In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm 

In [ ]:
REBUILD_DATA = False

class DogsVSCats:
    IMG_SIZE = 50
    CATS = "./data/PetImages/Cat"
    DOGS = "./data/PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    dog_count = 0
    cat_count = 0
    
    
    def build_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    if img is None:
                        print(f"Warning: {path} could not be read. Skipping this image.")
                        continue
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))  # pyright: ignore[reportCallIssue]
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    if label == self.CATS:
                        self.cat_count +=1
                    if label == self.DOGS:
                        self.dog_count +=1
                except Exception as e:
                    pass
        np.random.shuffle(self.training_data)
        np.save("training_data.npy",np.array(self.training_data, dtype=object), True, fix_imports=True)
        # print("CATS: ", self.cat_count)
        # print("DOGS: ", self.dog_count+ self.cat_count)

if REBUILD_DATA:
    dogs_vs_cats = DogsVSCats()
    dogs_vs_cats.build_training_data()

In [ ]:
training_data = np.load("training_data.npy", allow_pickle=True)
training_data[0]

In [ ]:
from torch._tensor import Tensor


from torch._tensor import Tensor
from torch.nn.modules.conv import Conv2d


import torch as T
import torch.nn as nn
import torch.nn.functional as F



class ConvolutionalNeuralNetWork(nn.Module):
    def __init__(self):
        super().__init__()
        self.convolutional_layer_one: Conv2d = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.convolutional_layer_two: Conv2d = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.convolutional_layer_three: Conv2d = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        
        # Initializing a final layer for distribution
        """
        The first argument -1 means 'for any size of tensor'.
        The second (hint 1) represent the entire batch of data.
        The 50 by 50 represents the size of images within the batch of data.
        """
        x: Tensor = T.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = 0
        self.layer_pooling(x)
        self.fully_connected_layer_one = nn.Linear(self._to_linear, 512)
        # Final layer
        self.fully_connected_layer_two = nn.Linear( 512, 2)
        
    def layer_pooling(self, layer: Tensor) -> Tensor:
        output_layer_one: Tensor=  F.max_pool2d(F.relu(self.convolutional_layer_one(layer)), (2,2))
        output_layer_two: Tensor=  F.max_pool2d(F.relu(self.convolutional_layer_two(output_layer_one)), (2,2))
        layer_pool: Tensor=  F.max_pool2d(F.relu(self.convolutional_layer_three(output_layer_two)), (2,2))

        if self._to_linear == 0:
            layer_one_shape = layer_pool[0].shape[0]
            layer_two_shape = layer_pool[0].shape[1]
            layer_three_shape = layer_pool[0].shape[2]
            product_of_layer_dimensions = (layer_one_shape * layer_two_shape * layer_three_shape)
            self._to_linear =  product_of_layer_dimensions
        return layer_pool
    
    def flattenFlattening(self, layer: Tensor) -> Tensor | None:
        layer_hat: Tensor = self.layer_pooling(layer)
        if self._to_linear != 0:
            layer_hat = layer_hat.view(-1, self._to_linear)
            return  layer_hat
        return None

    def forward(self, layer) -> Tensor | None:
        flatten_final_layer: Tensor | None = self.flattenFlattening(layer)
        if flatten_final_layer is not None:
            flatten_final_layer = F.relu(input=self.fully_connected_layer_one(flatten_final_layer))
            final_layer = self.fully_connected_layer_two(flatten_final_layer)
            return F.softmax(final_layer, dim=1)
neural_net: ConvolutionalNeuralNetWork = ConvolutionalNeuralNetWork()


In [ ]:
import torch.optim as optim
optimizer = optim.Adam(neural_net.parameters(), lr=0.001)
loss_func = nn.MSELoss()

images = Tensor(np.array([image[0] for image in training_data]))
images = images/255.0
y = Tensor([y[1] for y in training_data])

# The percentage of data we take from the set for sampling
sample_percent = 0.1
sample_end_index = int(len(images) * sample_percent)
Image_sample = images[:-sample_end_index]
y_sample = y[:-sample_end_index]

Image_test_sample = images[-sample_end_index:]
y_test = y[-sample_end_index:]

print(len(Image_sample))
print(len(Image_test_sample))



In [ ]:
BATCH_SIZE = 100
EPOCHS = 2
loss = 0
for epoch in range(EPOCHS):
    for index in tqdm(range(0, len(Image_sample), BATCH_SIZE)):
        batch_images = Image_sample[index: index+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = y_sample[index: index+BATCH_SIZE]
        neural_net.zero_grad()
        outputs = neural_net(batch_images)
        loss = loss_func(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Run our predictions

In [ ]:
correct = 0
total = 0
with T.no_grad():
    for index in tqdm(range(len(Image_test_sample))):
        real_class = T.argmax(y_test[index])
        net_out = neural_net(Image_test_sample[index].view(-1, 1, 50, 50))[0]
        
        predicted_class = T.argmax(net_out)
        if predicted_class == real_class:
            correct +=1
        total +=1
print(f"Accuracy: {(round(correct/total,3)*100)}%")

In [ ]:
T.cuda.is_available()

device = T.device("cuda" if T.cuda.is_available() else "cpu")
T.cuda.device_count()